
# IA Projeto01: Class Timetable CSP

**Objetivo:** Gerar horários de aulas para turmas de um curso universitário, respeitando restrições de professores, salas, horários e preferências.  

**Student** Fábio Mota - 9710

**Dataset:** `ClassTT_01_tiny.txt`

**Bibliotecas:** `python-constraint`, `collections`, `itertools`, `time`

**1 - Parser**

In [ ]:
import re

def parse_dataset(filename):
    """
    Lê o dataset e retorna dicionário estruturado.
    """
    with open(filename, 'r', encoding='utf-8') as f:
        lines = [line.strip() for line in f if line.strip()]

    data = {'cc': {}, 'dsd': {}, 'tr': {}, 'rr': {}, 'oc': {}, 'olw': []}
    section = None

    for line in lines:
        line = line.replace('—','-').strip()
        if line.lower().startswith('#cc'): section='cc'; continue
        elif line.lower().startswith('#dsd'): section='dsd'; continue
        elif line.lower().startswith('#tr'): section='tr'; continue
        elif line.lower().startswith('#rr'): section='rr'; continue
        elif line.lower().startswith('#oc'): section='oc'; continue
        elif line.lower().startswith('#olw'): section='olw'; continue
        elif line.lower().startswith('#head'): section='head'; continue

        if section=='head': continue
        parts = re.split(r'\s+', line)
        if not parts: continue

        if section=='cc':
            turma=parts[0]; cursos=parts[1:]; data['cc'][turma]=cursos
        elif section=='dsd':
            prof=parts[0]; cursos=parts[1:]
            for c in cursos: data['dsd'][c]=prof
        elif section=='tr':
            prof=parts[0]; slots=list(map(int, parts[1:])); data['tr'][prof]=slots
        elif section=='rr':
            curso,sala=parts; data['rr'][curso]=sala
        elif section=='oc':
            curso,idx=parts; data['oc'][curso]=int(idx)

    return data

# Testar parser
dataset = parse_dataset("ClassTT_01_tiny.txt")
from pprint import pprint
pprint(dataset)


**2 - Construção do problema CSP**

In [ ]:
# 2️⃣ Construção do problema CSP
from constraint import Problem
import itertools
from collections import defaultdict, Counter

SLOTS = list(range(1,21))  # 20 blocos
def day_of_slot(slot): return (slot-1)//4 + 1

def build_problem(data, rooms=None, lessons_per_course=2):
    """
    Cria o problema CSP com variáveis, domínio e restrições.
    """
    if rooms is None:
        rooms=["Lab01","RoomA","RoomB","RoomC","RoomD"]

    problem = Problem()
    cc = data['cc']
    course_to_class={c:cls for cls,courses in cc.items() for c in courses}
    course_list = sorted(course_to_class.keys())
    course_to_teacher = data.get('dsd', {})
    room_restrictions = data.get('rr', {})
    online_info = data.get('oc', {})

    variables_info = {}
    for c in course_list:
        for i in range(1, lessons_per_course+1):
            var = f"{c}_{i}"
            if c in room_restrictions:
                domain = [(s, room_restrictions[c]) for s in SLOTS]
            else:
                domain = [(s,r) for s in SLOTS for r in rooms]
            variables_info[var] = {
                'course': c,
                'class': course_to_class[c],
                'teacher': course_to_teacher.get(c),
                'index': i,
                'domain': domain
            }
            problem.addVariable(var, domain)

    vars_list=list(variables_info.keys())

    # 1) Nenhum slot/room repetido
    for v1,v2 in itertools.combinations(vars_list,2):
        problem.addConstraint(lambda a,b: a!=b, (v1,v2))

    # 2) Professor não pode ter 2 aulas ao mesmo slot
    for v1,v2 in itertools.combinations(vars_list,2):
        t1=variables_info[v1]['teacher']; t2=variables_info[v2]['teacher']
        if t1 and t1==t2: problem.addConstraint(lambda a,b: a[0]!=b[0], (v1,v2))

    # 3) Restrições de indisponibilidade de professor
    for var, meta in variables_info.items():
        teacher=meta['teacher']
        if teacher and teacher in data.get('tr', {}):
            forbidden=set(data['tr'][teacher])
            problem.addConstraint(lambda pair, forbidden=forbidden: pair[0] not in forbidden, (var,))

    # 4) No máximo 3 aulas/dia por turma
    class_to_vars=defaultdict(list)
    for var, meta in variables_info.items(): class_to_vars[meta['class']].append(var)
    for cls, cls_vars in class_to_vars.items():
        problem.addConstraint(lambda *assigned: all(v<=3 for v in Counter(day_of_slot(a[0]) for a in assigned).values()), tuple(cls_vars))

    # 5) Aulas online no mesmo dia
    online_vars=[f"{c}_{idx}" for c,idx in online_info.items() if f"{c}_{idx}" in variables_info]
    if online_vars:
        problem.addConstraint(lambda *assigned: len(set(day_of_slot(a[0]) for a in assigned))==1, tuple(online_vars))

    # 6) Cada curso: aulas não podem ser no mesmo slot
    for c in course_list:
        v1,v2=f"{c}_1",f"{c}_2"
        if v1 in variables_info and v2 in variables_info:
            problem.addConstraint(lambda a,b: a[0]!=b[0], (v1,v2))

    return problem, variables_info



**3 - Solver e Avaliação**

In [ ]:
from collections import defaultdict

def score_solution(sol, variables_info):
    """
    Calcula (soft constraints).
    """
    penalty=0
    def day_of_slot(slot): return (slot-1)//4+1

    # 1) Aulas do mesmo curso em dias diferentes
    for var, meta in variables_info.items():
        c = meta['course']
        v1,v2=f"{c}_1",f"{c}_2"
        if v1 in sol and v2 in sol and day_of_slot(sol[v1][0])==day_of_slot(sol[v2][0]):
            penalty+=10

    # 2) Turmas usando mais de 4 dias
    class_to_slots=defaultdict(list)
    for var,val in sol.items(): class_to_slots[variables_info[var]['class']].append(val[0])
    for slots in class_to_slots.values():
        days=set((s-1)//4+1 for s in slots)
        if len(days)>4: penalty+=5

    return penalty

def find_best_solution(problem, variables_info, max_solutions=50):
    best=None; best_score=float('inf'); sols_examined=0
    for sol in problem.getSolutionIter():
        sols_examined+=1
        sc=score_solution(sol, variables_info)
        if sc<best_score: best_score=sc; best=sol
        if sols_examined>=max_solutions: break
    return best, best_score, sols_examined

def print_solution(sol, variables_info):
    if sol is None: print("Sem solução"); return
    days_names={1:"Seg",2:"Ter",3:"Qua",4:"Qui",5:"Sex"}
    schedule=defaultdict(lambda: defaultdict(list))
    for var,val in sol.items():
        slot,room=val; cls=variables_info[var]['class']; course=variables_info[var]['course']
        pos=(slot-1)%4+1; day=(slot-1)//4+1
        schedule[cls][day].append((pos,course,room))
    for cls in sorted(schedule.keys()):
        print(f"=== Classe {cls} ===")
        for d in range(1,6):
            entries=sorted(schedule[cls].get(d,[]))
            if entries: print(f" {days_names[d]}: ", ", ".join(f"bloco{pos} {course} ({room})" for pos,course,room in entries))



**4 - Executar CSP e mostrar solução**

In [ ]:
dataset = parse_dataset("ClassTT_01_tiny.txt")
problem, variables_info = build_problem(dataset)
best, score, n_sols = find_best_solution(problem, variables_info)

print(f"Soluções examinadas: {n_sols}, pontuação: {score}")
print_solution(best, variables_info)
